#### Reading orders data from silver table

In [0]:
%sql
SELECT * FROM gizmobox.silver.orders_json;

##### 1. Access elements from json object

In [0]:
%sql
SELECT json_value.order_id,
        json_value.order_status,
        json_value.payment_method,
        json_value.total_amount,
        json_value.transaction_timestamp,
        json_value.customer_id,
        json_value.items
FROM gizmobox.silver.orders_json
ORDER by json_value.order_id

In [0]:
%sql
SELECT 
      json_value.order_id,
      json_value.order_status,
      json_value.payment_method,
      json_value.total_amount,
      json_value.transaction_timestamp,
      json_value.customer_id,
      array_distinct(json_value.items) as items
FROM gizmobox.silver.orders_json
ORDER BY json_value.order_id

##### 3. Explore arrays to get data in required format

In [0]:
%sql
SELECT 
      json_value.order_id,
      json_value.order_status,
      json_value.payment_method,
      json_value.total_amount,
      json_value.transaction_timestamp,
      json_value.customer_id,
      explode(array_distinct(json_value.items)) as item
FROM gizmobox.silver.orders_json
ORDER BY json_value.order_id

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tv_orders_exploded
AS
SELECT 
      json_value.order_id,
      json_value.order_status,
      json_value.payment_method,
      json_value.total_amount,
      json_value.transaction_timestamp,
      json_value.customer_id,
      explode(array_distinct(json_value.items)) as item
FROM gizmobox.silver.orders_json
ORDER BY json_value.order_id

In [0]:
%sql
SELECT * FROM tv_orders_exploded;

In [0]:
%sql
SELECT 
    order_id,
    order_status,
    payment_method,
    total_amount,
    transaction_timestamp,
    customer_id,
    item.item_id,
    item.name,
    item.price,
    item.quantity,
    item.category,
    item.details.brand,
    item.details.color
FROM tv_orders_exploded;
  

##### 4. Write data to silver table

In [0]:
%sql
CREATE OR REPLACE TABLE gizmobox.silver.orders
AS
SELECT 
    order_id,
    order_status,
    payment_method,
    total_amount,
    transaction_timestamp,
    customer_id,
    item.item_id,
    item.name,
    item.price,
    item.quantity,
    item.category,
    item.details.brand,
    item.details.color
FROM tv_orders_exploded;
  

In [0]:
%sql
SELECT * FROM 
gizmobox.silver.orders;